# LUME-services demo
In this notebook, we will configure LUME-services to use the service configuration used to launch our docker-compose services. Make sure you've completed all steps outlined in https://slaclab.github.io/lume-services/demo/.

In [1]:
import logging
logging.basicConfig(level=logging.INFO)  # Lets check the logs

## Configure services
LUME-services is packages with a configuration utility that reads environment variables and initializes services:

In [2]:
from lume_services import config
config.configure()

INFO:lume_services.config:Configuring LUME-services environment...
INFO:lume_services.config:Environment configured.


## if you're running this many time, creation will fail because of uniqueness... You can reset since this is a dev server

In [3]:
#model_db_service._reset()

## Create a model
The LUME-services Model provides an API to all model services and facilitates all model operations.

In [4]:
from lume_services.models import Model

model = Model.create_model(
    author = "Jackie Garrahan",
    laboratory = "slac",
    facility = "lcls",
    beampath = "cu_hxr",
    description = "test_model"
)
model

INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.author = :author_1 AND model.laboratory = :laboratory_1 AND model.facility = :facility_1 AND model.beampath = :beampath_1 AND model.description = :description_1
INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO model (author, laboratory, facility, beampath, description) VALUES (:author, :laboratory, :facility, :beampath, :description)
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO model (author, laboratory, facility, beampath, description) VALUES (:author, :laboratory, :facility, :beampath, :description)
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.model_id = :model_id_1


Model(metadata=Model(                     model_id=1,                     created=datetime.datetime(2022, 10, 1, 6, 48, 27),                     author='Jackie Garrahan'),                     laboratory='slac',                     facility='lcls',                     beampath='cu_hxr',                     description='test_model'                 ), deployment=None, results=None)

## Create a project
Workflows are organized by the Prefect scheduler into different projects. Below, we access the configured services directly (TODO create project registry utility)

In [5]:
model_db_service = config.context.model_db_service()
scheduling_service = config.context.scheduling_service()

In [6]:
### NOTE: The below cell will raise an error if run 2x

In [7]:
# create a project
project_name = model_db_service.store_project(
    project_name="test", description="my_description"
)
scheduling_service.create_project("test")

INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO project (project_name, description) VALUES (:project_name, :description)
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO project (project_name, description) VALUES (:project_name, :description)


You can now find this project in you Prefect UI at http://localhost:8080


![project](https://slaclab.github.io/lume-services/files/project_nav.png)

## Create a deployment for your model
Replace `source_path` with the path to your release tarball below:

In [8]:
source_path = "https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.44/my_model-0.0.44.tar.gz"
# populates local channel
model.store_deployment(source_path, project_name="test")

INFO:lume_services.models.model:installing package
INFO:lume_services.environment.solver:https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.44/my_model-0.0.44.tar.gz saved to /var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/tmp4c5jbhpr/my_model-0.0.44.tar.gz
INFO:lume_services.environment.solver:Version 0.0.44 of my_model already installed.
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT deployment.sha256, deployment.deployment_id, deployment.version, deployment.deploy_date, deployment.package_import_name, deployment.asset_dir, deployment.source, deployment.image, deployment.is_live, deployment.model_id 
FROM deployment 
WHERE deployment.model_id = :model_id_1 AND deployment.version = :version_1
INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO deployment (version, package_import_name, asset_dir, source, sha256, image, is_live, model_id) VALUES (:version, :package_import_name, :asset_dir, :source, :sha256, :image, :is_live, :m

Flow URL: http://localhost:8080/default/flow/c13c5222-b967-404c-bf41-5ff5c1f0f38e
 └── ID: 8352b84e-ae5f-4e86-9143-921b0eb6d37d
 └── Project: test
 └── Labels: ['lume-services']


## Run the Prefect workflow directly

In [9]:
flow_run = model.deployment.flow.prefect_flow.run(**{
                        "input1": 1, 
                        "input2": 2, 
                        "filename": "test_file.txt", 
                        "filesystem_identifier":"local"
    }
)

[2022-09-30 23:48:28-0700] INFO - prefect.FlowRunner | Beginning Flow run for 'my-model'


INFO:prefect.FlowRunner:Beginning Flow run for 'my-model'


[2022-09-30 23:48:28-0700] INFO - prefect.TaskRunner | Task 'configure_lume_services': Starting task run...


INFO:prefect.TaskRunner:Task 'configure_lume_services': Starting task run...
INFO:lume_services.config:Configuring LUME-services environment...
INFO:lume_services.config:Environment configured.


[2022-09-30 23:48:29-0700] INFO - prefect.TaskRunner | Task 'configure_lume_services': Finished task run for task with final state: 'Success'


...

INFO:prefect.TaskRunner:Task 'save_db_result': Starting task run...


[2022-09-30 23:48:29-0700] INFO - prefect.TaskRunner | Task 'save_db_result': Finished task run for task with final state: 'Success'


INFO:prefect.TaskRunner:Task 'save_db_result': Finished task run for task with final state: 'Success'


[2022-09-30 23:48:29-0700] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


INFO:prefect.FlowRunner:Flow run SUCCESS: all reference tasks succeeded


## Run the workflow inside the service cluster
We can use the model interface to directly deploy workflows. When sourcing our environment (`docs/examples/demo.env`), we defined a mount point for the file system using the alias `/lume-services/data`. Let's kick off this workflow and save the file output to that directory. 
After running the next cell, you'll be able to see the running container in your docker desktop and examine the flow using the Prefect UI at http://localhost:8080/default?flows.

In [10]:
model.run(
    parameters={
        "input1": 1, 
        "input2": 2, 
        "filename": "/lume-services/data/test_file.txt", 
        "filesystem_identifier":"mounted"}
)

In [ ]:
model.deployment.flow

# Get results:

In [ ]:
results = model.get_results()
results

In [ ]:
results_df = model.get_results_df()
results_df

## Load model using model id
Once your model has been registered, you can use the `Model` api object to interact with your model without running the above registration steps. Let's load a new model object using the model_id we registered above.

In [ ]:
from importlib import reload 
import lume_services
reload(lume_services.models)
from lume_services.models import Model

model_id = model.metadata.model_id
loaded_model = Model(model_id=model_id)

In [ ]:
loaded_model.metadata

## Load existing model object
Loading a model using the load_deployment method without passing a deployment_id will load the latest deployment registered for the model.

In [ ]:
loaded_model.load_deployment()
loaded_model.deployment

In [ ]:
results = loaded_model.run_and_return(
    collection="test",
    parameters={
                    "input1": 1.0, 
                    "input2": 2.0, 
                    "filename": "/lume-services/data/test_file.txt", 
                    "filesystem_identifier":"local"
    },
    task_name="save_db_result" # Want to get the result from the save_db_result task
)
res